In [14]:
from sklearn.model_selection import train_test_split
import random
import matplotlib.pyplot as plt
import numpy as np
import cv2

### Carga de Datos

Nota: A partir del c13 al c18 son duplicados

In [15]:

conjuntoImgs = []
conjuntoGris = []
conjuntoBin = []
y = []

for i in range(1, 19):
    for j in range(10):
        ruta = f'Prueba/C{i}/N{j}.JPG'
        temp = cv2.imread(ruta)

        temp = cv2.resize(temp, (32, 64))
        temp_RGB = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
        conjuntoImgs.append(temp)
        
        temp_gris = cv2.cvtColor(temp_RGB, cv2.COLOR_RGB2GRAY)
        conjuntoGris.append(temp_gris)
        
        _, temp_bin = cv2.threshold(temp_gris, 127, 1, cv2.THRESH_BINARY)
        conjuntoBin.append(temp_bin)
    
        match(j):
            case 0: y.append([1,0,0,0,0,0,0,0,0,0])
            case 1: y.append([0,1,0,0,0,0,0,0,0,0])
            case 2: y.append([0,0,1,0,0,0,0,0,0,0])
            case 3: y.append([0,0,0,1,0,0,0,0,0,0])
            case 4: y.append([0,0,0,0,1,0,0,0,0,0])
            case 5: y.append([0,0,0,0,0,1,0,0,0,0])
            case 6: y.append([0,0,0,0,0,0,1,0,0,0])
            case 7: y.append([0,0,0,0,0,0,0,1,0,0])
            case 8: y.append([0,0,0,0,0,0,0,0,1,0])
            case 9: y.append([0,0,0,0,0,0,0,0,0,1])

# Convertir las listas en arrays de numpy
conjuntoImgs = np.array(conjuntoImgs)
conjuntoGris = np.array(conjuntoGris)
conjuntoBin = np.array(conjuntoBin)
y = np.array(y)



### Declaración de funciones

In [16]:
# Definiendo funcion de activacion

def sigmoid(x):
	return 1/(1 + np.exp(-x))

def f_forward(x, w1, w2):
	# Oculta
	z1 = x.dot(w1)# Entrada de la capa 1 (oculta)
	a1 = sigmoid(z1)# Salia para la entrada de la capa 2
	
	# Capa de salida
	z2 = a1.dot(w2)# entrada de la capa 2 o de salida
	a2 = sigmoid(z2)# salida de la capa
	return(a2)

# Generacion aleatoria de pesos
def generate_wt(x, y):
	return np.random.randn(x, y)
	
# cálculo de la perdida con mean square error(MSE)
def loss(out, Y):
	s =(np.square(out-Y))
	s = np.sum(s)/len(Y)
	return(s)

# Back propagation of error 
def back_prop(x, y, w1, w2, alpha):
	
	# capa oculta
	z1 = x.dot(w1)# entrada de la capa 1
	a1 = sigmoid(z1)# entrada de la capa 2 (salida)
	
	# capa de salida
	z2 = a1.dot(w2)# entrada de la capa de salida
	a2 = sigmoid(z2)# salida de la capa de salida
	# error en la capa de salida
	d2 =(a2-y)
	d1 = np.multiply((w2.dot((d2.transpose()))).transpose(), 
								(np.multiply(a1, 1-a1)))

	# Gradiente para w1 y w2
	w1_adj = x.transpose().dot(d1)
	w2_adj = a1.transpose().dot(d2)
	
	# Actualizando parámetros
	w1 = w1-(alpha*(w1_adj))
	w2 = w2-(alpha*(w2_adj))
	
	return(w1, w2)

def train(x, Y, w1, w2, alpha = 0.01, epoch = 10):
	acc =[]
	losss =[]
	for j in range(epoch):
		l =[]
		for i in range(len(x)):
			out = f_forward(x[i], w1, w2)
			l.append((loss(out, Y[i])))
			w1, w2 = back_prop(x[i], Y[i], w1, w2, alpha)
		print("epochs:", j + 1, "======== acc:", (1-(sum(l)/len(x)))*100) 
		acc.append((1-(sum(l)/len(x)))*100)
		losss.append(sum(l)/len(x))
	return(acc, losss, w1, w2)

def predict(x, w1, w2):
	Out = f_forward(x, w1, w2)
	k = np.argmax(Out)  # Índice del valor máximo
	# print(f"El numero en la imagen es: {r}.")
	# plt.figure()
	# plt.imshow(x.reshape(64, 32))
	# plt.show()
	return k

def predict_RGB(x, w1, w2):
	Out = f_forward(x, w1, w2)
	k = np.argmax(Out)  # Índice del valor máximo
	# print(f"El numero en la imagen es: {r}.")
	# plt.figure()
	# plt.imshow(x.reshape(64, 32, 3))
	# plt.show()
	return k

### Partición de los datos

In [17]:
X_train_RGB, X_test_RGB, Y_train_RGB, Y_test_RGB = train_test_split(conjuntoImgs, y, test_size = 0.2)
X_train_Gris, X_test_Gris, Y_train_Gris, Y_test_Gris = train_test_split(conjuntoGris, y, test_size = 0.2)
X_train_Bin, X_test_Bin, Y_train_Bin, Y_test_Bin = train_test_split(conjuntoBin, y, test_size = 0.2)

X_train_RGB = X_train_RGB.reshape(144, 1, -1)  
X_test_RGB = X_test_RGB.reshape(36, 1, -1)  
Y_train_RGB = Y_train_RGB.reshape(144, 1, -1)
Y_test_RGB = Y_test_RGB.reshape(36, 1, -1)

X_train_Gris = X_train_Gris.reshape(144, 1, -1)  
X_test_Gris = X_test_Gris.reshape(36, 1, -1)  
Y_train_Gris = Y_train_Gris.reshape(144, 1, -1)
Y_test_Gris = Y_test_Gris.reshape(36, 1, -1)

X_train_Bin = X_train_Bin.reshape(144, 1, -1)  
X_test_Bin = X_test_Bin.reshape(36, 1, -1)
Y_train_Bin = Y_train_Bin.reshape(144, 1, -1)
Y_test_Bin = Y_test_Bin.reshape(36, 1, -1)  

### Entrenamiento de las redes

##### RBG

In [18]:
w1_RGB = generate_wt(6144, 48)
w2_RGB = generate_wt(48, 10)
print(w1_RGB, "\n\n", w2_RGB)

[[-0.12652754 -0.17332528  0.19147315 ...  1.05266629 -0.07450735
  -0.86790803]
 [ 1.44306346  1.1344635  -0.76772723 ... -0.51041723  0.65266889
  -0.09898659]
 [-0.19817288  0.16901893  1.10766142 ... -0.67081485  0.77671257
  -0.12665121]
 ...
 [ 0.73261621  0.07518463 -2.00589011 ... -0.00418581  0.30619847
  -0.63980962]
 [ 1.47989478  1.06333215  0.50802244 ... -0.69411422  0.07168891
   0.41486053]
 [-1.59756077  1.15279394 -0.52503458 ...  0.24690197  1.39868886
  -0.43418818]] 

 [[ 2.33242888e+00  2.32832308e+00  1.15088967e-01  9.69694872e-01
  -1.94193017e+00  1.32455254e+00  1.86877481e-01  2.88174288e-01
   4.53909516e-01  1.03780505e+00]
 [-1.50375049e-01  1.30704141e+00  1.08364470e+00  3.08504039e-01
  -9.57969352e-01  7.83917147e-01  4.47007828e-02 -1.88649274e-01
   5.12393319e-01 -1.14909311e-01]
 [-1.73620791e-01  1.18684488e+00  1.10523360e-01 -8.90621664e-01
  -5.76906641e-01  1.39973530e+00  1.87955367e+00 -1.07673194e-01
  -1.20991840e+00 -1.55940202e-02]
 [-2

In [19]:
acc_RGB, losss_RGB, w1_RGB, w2_RGB = train(X_train_RGB, Y_train_RGB, w1_RGB, w2_RGB, 0.1, 100)

C:\Users\abiab\AppData\Local\Temp\ipykernel_32744\3133336208.py:4: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-x))


epochs: 1 ======== acc: 6.577579453829663
epochs: 2 ======== acc: 55.18409234578958
epochs: 3 ======== acc: 71.04137553573764
epochs: 4 ======== acc: 77.43154666469476
epochs: 5 ======== acc: 80.54039773720393
epochs: 6 ======== acc: 82.2691445955368
epochs: 7 ======== acc: 83.30855123063859
epochs: 8 ======== acc: 83.96896881688198
epochs: 9 ======== acc: 84.4079568791794
epochs: 10 ======== acc: 84.7113492885125
epochs: 11 ======== acc: 84.92827488904862
epochs: 12 ======== acc: 85.08802715232734
epochs: 13 ======== acc: 85.2087250389242
epochs: 14 ======== acc: 85.30195905575366
epochs: 15 ======== acc: 85.37537535124076
epochs: 16 ======== acc: 85.43416113503656
epochs: 17 ======== acc: 85.48192549136786
epochs: 18 ======== acc: 85.52123722238598
epochs: 19 ======== acc: 85.55396241303667
epochs: 20 ======== acc: 85.58148177054058
epochs: 21 ======== acc: 85.60483389105987
epochs: 22 ======== acc: 85.62481176460054
epochs: 23 ======== acc: 85.64202908400534
epochs: 24 ======== acc:

#### Escala de Grises

In [24]:
w1_Gris = generate_wt(2048, 64)
w2_Gris = generate_wt(64, 10)
print(w1_Gris, "\n\n", w2_Gris)

[[-2.72366862 -0.34716058 -0.31790346 ... -1.22686657  0.02070111
   1.67730418]
 [ 0.37600694 -1.6870124   1.43849784 ...  2.49113066  1.0966145
  -0.34054077]
 [-0.14349185  0.0127097  -1.23957359 ...  1.2994392   0.38005499
   0.2675503 ]
 ...
 [-0.7206952  -0.99959139 -1.06075935 ...  1.42408858  0.04158243
  -0.48980618]
 [-0.10147291  1.10759419 -0.41614325 ...  2.1922405  -1.96516461
   0.22673445]
 [-1.76048681 -0.96396849  0.0486751  ... -0.04327725  0.55976438
   0.52711981]] 

 [[ 2.89126682e-01  2.82941853e-01  5.29480209e-02  2.19222413e-01
   2.85418411e-01 -5.14971375e-01 -5.22560103e-01 -4.28326515e-01
   1.23481935e+00  5.96965770e-02]
 [-9.80376260e-01  5.00092365e-01  3.45589355e-01 -1.73433708e+00
   5.76392216e-01  6.69008999e-01  5.68989640e-01 -1.96763449e+00
   3.47958096e-01  1.52335451e+00]
 [-2.14894193e+00 -1.31871924e+00  4.58466013e-01 -9.54631363e-01
   1.05573719e+00 -1.08705347e-01  3.99794831e-01 -3.36313918e-01
  -1.39881855e+00 -3.11983069e-01]
 [-5.

In [25]:
acc_Gris, losss_Gris, w1_Gris, w2_Gris = train(X_train_Gris, Y_train_Gris, w1_Gris, w2_Gris, 0.1, 100)

C:\Users\abiab\AppData\Local\Temp\ipykernel_32744\3133336208.py:4: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-x))


epochs: 1 ======== acc: 6.9960023094584045
epochs: 2 ======== acc: 66.0957295863851
epochs: 3 ======== acc: 86.46016684832007
epochs: 4 ======== acc: 93.1731018465937
epochs: 5 ======== acc: 95.89676004432629
epochs: 6 ======== acc: 97.27748971441498
epochs: 7 ======== acc: 98.07279626519177
epochs: 8 ======== acc: 98.5695810754138
epochs: 9 ======== acc: 98.89881544299558
epochs: 10 ======== acc: 99.12736030052713
epochs: 11 ======== acc: 99.29206931200977
epochs: 12 ======== acc: 99.41449621803004
epochs: 13 ======== acc: 99.50787461655098
epochs: 14 ======== acc: 99.58066941160006
epochs: 15 ======== acc: 99.6384879921144
epochs: 16 ======== acc: 99.68515864472667
epochs: 17 ======== acc: 99.72336453041729
epochs: 18 ======== acc: 99.75502998025112
epochs: 19 ======== acc: 99.78156341242256
epochs: 20 ======== acc: 99.80401441204964
epochs: 21 ======== acc: 99.82317786315069
epochs: 22 ======== acc: 99.83966453585904
epochs: 23 ======== acc: 99.85394990503804
epochs: 24 ======== acc

#### Binario

In [32]:
w1_Bin = generate_wt(2048, 128)
w2_Bin = generate_wt(128, 10)
print(w1_Bin, "\n\n", w2_Bin)

[[ 0.91856934 -0.57226888  0.37006842 ... -0.84889677 -0.59472789
  -0.25310263]
 [ 0.98709018  0.47943673 -0.87256321 ...  0.97275081  0.23525709
  -0.47163572]
 [ 0.74084851  0.20221219  0.61376344 ...  2.58763389  0.73091794
  -1.06692252]
 ...
 [-0.0938236  -1.6649068   1.38757948 ...  0.50328218 -0.19405131
  -1.38437891]
 [-1.6244316  -0.44794088 -0.93651948 ... -1.42843088  0.54493615
  -0.18392062]
 [ 0.0148761   1.74698738  1.19765976 ... -0.90628312 -0.95365331
  -0.72281393]] 

 [[-0.40848405 -0.45142494  1.02253445 ... -0.75252171 -1.28686878
  -1.03160981]
 [ 0.03821759 -1.03563773  1.01901397 ... -0.5526558   1.45733375
  -0.54810307]
 [ 0.01887056 -0.70691696 -0.69462832 ... -0.95092425  0.80353384
  -1.41584975]
 ...
 [-1.27028609 -1.05567304 -0.75537702 ...  0.48184724 -0.73120951
  -0.38060683]
 [-0.2231946  -1.7750959  -0.81715392 ... -0.57941496 -0.11087761
  -0.54498787]
 [ 0.06686281 -0.79456792  0.48904582 ...  0.47254787 -1.69268557
  -0.73909262]]


In [33]:
acc_Bin, losss_Bin, w1_Bin, w2_Bin = train(X_train_Bin, Y_train_Bin, w1_Bin, w2_Bin, 0.01, 100)

epochs: 1 ======== acc: 9.014418276958525
epochs: 2 ======== acc: 58.03453506964562
epochs: 3 ======== acc: 70.78843052145996
epochs: 4 ======== acc: 80.81185426352032
epochs: 5 ======== acc: 87.31874842464002
epochs: 6 ======== acc: 91.28902568806521
epochs: 7 ======== acc: 94.00258680589255
epochs: 8 ======== acc: 95.74290266255299
epochs: 9 ======== acc: 96.78647736972972
epochs: 10 ======== acc: 97.48778250288338
epochs: 11 ======== acc: 97.9586482985434
epochs: 12 ======== acc: 98.27413710497879
epochs: 13 ======== acc: 98.51506389918147
epochs: 14 ======== acc: 98.70446874847948
epochs: 15 ======== acc: 98.85668748335122
epochs: 16 ======== acc: 98.9813102270796
epochs: 17 ======== acc: 99.08504361717894
epochs: 18 ======== acc: 99.1726988162951
epochs: 19 ======== acc: 99.24774368669563
epochs: 20 ======== acc: 99.31261237642454
epochs: 21 ======== acc: 99.36908156485627
epochs: 22 ======== acc: 99.41855632180726
epochs: 23 ======== acc: 99.46217835079717
epochs: 24 ======== acc

### Acuary Final

#### RGB

In [39]:
pred = []
real = []
for tx, ty in zip(X_test_RGB, Y_test_RGB):
    pred.append(predict_RGB(tx, w1_RGB, w2_RGB))
    real.append(np.argmax(ty))

error = 0
for p, r in zip(pred, real):
    if p != r:
        error += 1

aciertos = ((len(pred) - error) / len(pred)) * 100
print(f'{aciertos:.2f} %')
print(pred)
print(real)

83.33 %
[8, 8, 0, 2, 0, 7, 7, 4, 8, 3, 9, 8, 0, 4, 6, 3, 2, 9, 2, 3, 7, 3, 3, 8, 7, 7, 4, 8, 4, 2, 8, 8, 3, 2, 9, 5]
[1, 1, 0, 2, 0, 7, 7, 4, 1, 3, 9, 1, 0, 4, 6, 3, 2, 9, 2, 3, 7, 3, 3, 1, 7, 7, 4, 8, 4, 2, 8, 1, 3, 2, 9, 5]


C:\Users\abiab\AppData\Local\Temp\ipykernel_32744\3133336208.py:4: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-x))


#### Escala de Grises

In [35]:
pred = []
real = []
for tx, ty in zip(X_test_Gris, Y_test_Gris):
    pred.append(predict(tx, w1_Gris, w2_Gris))
    real.append(np.argmax(ty))

error = 0
for p, r in zip(pred, real):
    if p != r:
        error += 1

aciertos = ((len(pred) - error) / len(pred)) * 100
print(f'{aciertos} %')
print(pred)
print(real)

100.0 %
[8, 7, 9, 7, 3, 2, 6, 5, 6, 8, 1, 3, 9, 0, 0, 3, 9, 8, 6, 0, 0, 7, 3, 1, 1, 9, 9, 5, 3, 1, 1, 3, 7, 4, 1, 1]
[8, 7, 9, 7, 3, 2, 6, 5, 6, 8, 1, 3, 9, 0, 0, 3, 9, 8, 6, 0, 0, 7, 3, 1, 1, 9, 9, 5, 3, 1, 1, 3, 7, 4, 1, 1]


C:\Users\abiab\AppData\Local\Temp\ipykernel_32744\3133336208.py:4: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-x))


#### Binario

In [36]:
pred = []
real = []
for tx, ty in zip(X_test_Bin, Y_test_Bin):
    pred.append(predict(tx, w1_Bin, w2_Bin))
    real.append(np.argmax(ty))

error = 0
for p, r in zip(pred, real):
    if p != r:
        error += 1

aciertos = ((len(pred) - error) / len(pred)) * 100
print(f'{aciertos} %')
print(pred)
print(real)

100.0 %
[3, 3, 4, 6, 5, 2, 0, 7, 5, 4, 7, 0, 0, 4, 7, 1, 4, 0, 8, 3, 1, 6, 8, 0, 1, 4, 2, 4, 8, 9, 7, 7, 8, 1, 0, 0]
[3, 3, 4, 6, 5, 2, 0, 7, 5, 4, 7, 0, 0, 4, 7, 1, 4, 0, 8, 3, 1, 6, 8, 0, 1, 4, 2, 4, 8, 9, 7, 7, 8, 1, 0, 0]
